In [68]:
from pathlib import Path
from matplotlib import pyplot as plt
import pandas as pd
import regex as re

# Importing selenium components
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common import TimeoutException
import sys


In [43]:
# Setting up the selenium webdriver
options = Options()
options.add_argument('--headless=new')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)


In [44]:
csv_path = Path('Resources/ASX_Listed_Companies_25-03-2024_02-42-20_AEDT.csv')
company_symbols = pd.read_csv(csv_path)
company_symbols

# create a scraped_data dataframe with symbols as columns

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D,1414 DEGREES LIMITED,Capital Goods,12/09/2018,14051943
1,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,14196980
2,1AE,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,21039989
3,1AG,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,3482763
4,1AI,ALGORAE PHARMACEUTICALS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",01/09/2004,14952172
...,...,...,...,...,...
1999,ZLD,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,7829537
2000,ZMI,ZINC OF IRELAND NL,Materials,18/09/2007,2770876
2001,ZMM,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,3942018
2002,ZNC,ZENITH MINERALS LIMITED,Materials,29/05/2007,30304756


In [73]:
query_start = 'https://au.finance.yahoo.com/quote/';
query_end = '?.tsrc=fin-srch'

# This is to select all items in list
# for i in len(company_symbols):
#     query_URL = f"{query_start}{company_symbols['ASX code'][i]}{query_end}"
#     soup = BeautifulSoup(query_URL, 'html.parser')
#     for article in soup.find_all('li class="js-stream-content Pos(r)"'):
#         link = article.get('href')
#         blurb = article.get('p')
#         title = article.get('a')

article_dict = {}
query_URL = "https://au.finance.yahoo.com/quote/IPH.AX?.tsrc=fin-srch"
driver.get(query_URL)

articles = driver.find_elements(By.CSS_SELECTOR, 'li[class="js-stream-content Pos(r)"]')

# The following code parses the raw html from selenium to a regex patern which extracts the text
for article in articles:
    raw_html = article.get_attribute('outerHTML')

    # Creating regex patterns
    title_pattern = r'(?:</u>)(.*?)(?:</a>)'
    blurb_pattern = r'(?:\(0\)">)(.*?)(?:</p>)'
    link_pattern = r'(?:href=")(.*?)(?: data-uuid)'

    # Performing a regex search
    title = re.search(title_pattern, raw_html)
    blurb = re.search(blurb_pattern, raw_html)
    link = re.search(link_pattern, raw_html)
    article_dict[title.group(1)] = {blurb.group(1):link.group(1)}
    

print(article_dict)

{'The past year for IPH (ASX:IPH) investors has not been profitable': {'Passive investing in an index fund is a good way to ensure your own returns roughly match the overall market. While...': '/news/past-iph-asx-iph-investors-230510741.html"'}, "We Wouldn't Be Too Quick To Buy IPH Limited (ASX:IPH) Before It Goes Ex-Dividend": {"Some investors rely on dividends for growing their wealth, and if you're one of those dividend sleuths, you might be...": '/news/wouldnt-too-quick-buy-iph-233604946.html"'}, 'IPH Limited (ASX:IPH) Shares Could Be 47% Below Their Intrinsic Value Estimate': {"Key Insights IPH's estimated fair value is AU$13.28 based on 2 Stage Free Cash Flow to Equity IPH is estimated to be...": '/news/iph-limited-asx-iph-shares-002124589.html"'}, "With 49% stake, IPH Limited (ASX:IPH) seems to have captured institutional investors' interest": {"Key Insights Institutions' substantial holdings in IPH implies that they have significant influence over the company's...": '/news/49-s

In [ ]:
# Closing the webdriver
driver.quit()